In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import when
from pyspark.sql.types import IntegerType
import datetime
import os

master = "spark://zy-ubuntu:7077"  
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--master {master} --driver-memory 4g --total-executor-cores 8 --executor-memory 8g --packages org.postgresql:postgresql:42.1.1 pyspark-shell'

In [2]:
spark = SparkSession.builder \
    .appName("title crew") \
    .getOrCreate()

# spark.sparkContext.getConf().getAll()

In [3]:
title_principals_df = spark.read.csv("title.principals.tsv", sep=r'\t', header=True)
title_principals_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- nconst: string (nullable = true)
 |-- category: string (nullable = true)
 |-- job: string (nullable = true)
 |-- characters: string (nullable = true)



In [4]:
# title_principals_df.show()

In [5]:
title_crew_df = title_principals_df.filter(
        (F.col('category') != 'self') &
        (F.col('category') != 'actor') & 
        (F.col('category') != 'actress'))

title_crew_df = title_crew_df.drop('job').drop('characters').drop('ordering')
title_crew_df = title_crew_df.withColumnRenamed('category', 'role')
# title_crew_df.select('role').distinct().show()     

# title_crew_df.show()

In [6]:
# read tsv file into df
name_basics_df = spark.read.csv("name.basics.tsv", sep=r'\t', header=True)

# rename column
name_basics_df = name_basics_df.withColumnRenamed('primaryName', 'name')

# calculate age from birth year and death year
name_basics_df = name_basics_df.withColumn("age", when((F.col("birthYear") != '\\N') & (F.col("deathYear") != '\\N'), (F.col('deathYear').cast(IntegerType()) - F.col('birthYear').cast(IntegerType()))).when((F.col("birthYear") != '\\N') & (F.col("deathYear") == '\\N'), (datetime.datetime.now().year - F.col('birthYear')).cast(IntegerType())).otherwise(None))

# create is_alive column based on conditions
is_alive_col = F.when(
    (F.col("birthYear") != '\\N') & (F.col("deathYear") != '\\N'), False
).when((F.col("birthYear") != '\\N') & (F.col("deathYear") == '\\N'), True).otherwise(None)

name_basics_df = name_basics_df.withColumn('is_alive', is_alive_col)

# drop unused columns
name_basics_df_dropped = name_basics_df.drop('primaryProfession', 'knownForTitles', 'birthYear', 'deathYear')

In [7]:
df_crew = title_crew_df.join(name_basics_df_dropped, ['nconst'])
# df_crew.show()

In [8]:
df_upload = df_crew.drop('tconst').drop('role').drop_duplicates(['nconst'])

In [9]:
df_upload.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_crew',
      user='admin',
      password='password'
      ).mode('append').save()

In [10]:
title_desc_df = spark.read.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_title_desc',
      user='admin',
      password='password'
      ).load()

In [11]:
title_desc_id_df = title_desc_df.select('id', 'tconst')
title_desc_id_df = title_desc_id_df.withColumnRenamed('id', 'title_id')
# title_desc_id_df.show()

In [12]:
crew_from_pg_df = spark.read.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='dim_crew',
      user='admin',
      password='password'
      ).load()
crew_from_pg_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: short (nullable = true)
 |-- is_alive: boolean (nullable = true)
 |-- nconst: string (nullable = true)



In [13]:
df_crew.printSchema()
title_desc_id_df.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- tconst: string (nullable = true)
 |-- role: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_alive: boolean (nullable = true)

root
 |-- title_id: integer (nullable = true)
 |-- tconst: string (nullable = true)



In [14]:
crew_id_df = crew_from_pg_df.select('id', 'nconst')
crew_id_df = crew_id_df.withColumnRenamed('id', 'crew_id')

In [15]:
df_crew_composite_table = df_crew.select('nconst', 'tconst', 'role')

In [16]:
df_tmp = crew_id_df.join(df_crew_composite_table, ['nconst'])

In [17]:
df_titles_crew = df_tmp.join(title_desc_id_df, ['tconst'])
df_titles_crew = df_titles_crew.drop('nconst').drop('tconst')
# df_titles_crew.show()

In [18]:
# df_titles_crew.count()

In [19]:
df_titles_crew.write.format('jdbc').options(
      url='jdbc:postgresql://localhost:5433/imdb',
      driver='org.postgresql.Driver',
      dbtable='titles_crew',
      user='admin',
      password='password'
      ).mode('append').save()